In [86]:
################################################################################
# Load dataset and split it into training and test set
################################################################################

import pandas as pd
import os
from tabulate import tabulate

sample_size = 10000

# Load dateset
df = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2.csv')

# Split dataset according to attack type
normal_df = df[df['Attack_label'] == 0]
attack_df = df[df['Attack_label'] == 1]

# Drop columns
normal_df = normal_df.drop(columns=['Attack_label', 'Attack_type'])
attack_df = attack_df.drop(columns=['Attack_label', 'Attack_type'])

# Split dataset into training and test set
normal_df_train = normal_df.sample(frac=0.8, random_state=42)
normal_df_test = normal_df.drop(normal_df_train.index)
attack_df_train = attack_df.sample(frac=0.8, random_state=42)
attack_df_test = attack_df.drop(attack_df_train.index)

# Print dataset sizes in a table
data = [
    ["Normal", normal_df.shape[0], normal_df_train.shape[0], normal_df_test.shape[0]],
    ["Attack", attack_df.shape[0], attack_df_train.shape[0], attack_df_test.shape[0]]
]
print(tabulate(data, headers=["Atack type", "Total", "Train", "Test"], tablefmt="grid"))

+--------------+---------+---------+--------+
| Atack type   |   Total |   Train |   Test |
+==============+=========+=========+========+
| Normal       |    5000 |    4000 |   1000 |
+--------------+---------+---------+--------+
| Attack       |    5000 |    4000 |   1000 |
+--------------+---------+---------+--------+


In [ ]:
# Generate Feature Importance
################################################################################

import os
import dotenv
import time
import numpy as np
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze the differences between normal and attack entries by comparing corresponding fields.
Output top 5 important features that can be used to filter an entry as either normal or attack.
Output only in the Python list structure.

Feature Names:
```{feature_names}```

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```

Example output:
['feature1', 'feature2', 'feature3', 'feature4', 'feature5']
"""
prompt = PromptTemplate(template=template, input_variables=["feature_names", "normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
model_name = "gpt-4o"
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)
# model_name = "gemini-1.5-pro"
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=10)['documents'][0]

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=10)['documents'][0]

completions = []
for i in range(10):
    completion = chain.invoke({
        "feature_names": normal_df_train.columns.to_list(),
        "normal_entries": ",\n".join([f"{doc} --> normal" for doc in normal_documents]),
        "attack_entries": ",\n".join([f"{doc} --> attack" for doc in attack_documents])
        })
    completions.append(completion.content)
    print(completion.content)
    time.sleep(10)

with open(f"results/feature-importance-{sample_size}-llm-{model_name}.txt", "a") as f:
    f.write("\n".join(completions))

In [29]:
# Generate Rules
################################################################################

import os
import dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze the differences between normal and attack entries by comparing corresponding fields.
Generate 5 simple and deterministic rules for top 5 important features to filter an entry as either normal or attack. 
Output only in the JSON format with the structure: 
{{'feature1': 'rule', 'feature2': 'rule', ..., 'feature5': 'rule'}}.

Feature Names:
```{feature_names}```

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```
"""
prompt = PromptTemplate(template=template, input_variables=["feature_names", "normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
model_name = "gpt-4o"
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)
# model_name = "gemini-1.5-pro"
# llm = ChatAnthropic(model='claude-3-opus-20240229')
# model_name = "claude-3-opus-20240229"
# chain = prompt
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db3-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=10)['documents']

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=10)['documents']

completion = chain.invoke({
    "feature_names": normal_df_train.columns.to_list(),
    "normal_entries": ",\n".join([f"{doc} --> normal" for doc in normal_documents]),
    "attack_entries": ",\n".join([f"{doc} --> attack" for doc in attack_documents])
    })

# print(completion)
print(completion.content)

# with open(f"results/generated-rules-{sample_size}-llm-{model_name}.txt", "a") as f:
#     f.write(completion.content)

c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


```json
{
  "tcp.dstport": "if tcp.dstport == 80.0 then attack else normal",
  "tcp.flags": "if tcp.flags == 18.0 then attack else normal",
  "tcp.options": "if tcp.options != '0' then attack else normal",
  "tcp.ack": "if tcp.ack == 1.0 then attack else normal",
  "tcp.len": "if tcp.len > 100.0 then attack else normal"
}
```


In [39]:
# Evaluate generated rule
################################################################################

from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

datasets = {"normal": normal_df_test, "attack": attack_df_test}
y_pred = []
y_true = []
for attack_type, dataset in datasets.items():
    test_set_size = dataset.shape[0]
    for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        predicted_attack_types = []
        # print(type(dataset.iloc[i]['mqtt.msgtype']))
        # predicted_attack_types.append("normal" if float(dataset.iloc[i]['dns.qry.name.len']) < 50 else "attack")
        # predicted_attack_types.append("normal" if float(dataset.iloc[i]['icmp.seq_le']) < 1000 else "attack")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['dns.retransmission'] == '0' else "attack")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['mqtt.protoname'] != 'MQTT' else "attack")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['mqtt.msg'] == '0' else "attack")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['mqtt.msgtype'] in [1,2,3] else "attack")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['mqtt.conack.flags'] == '0' else "attack")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['mqtt.hdrflags'] == '0' else "attack")
        predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.ack'] == 1 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.connection.fin'] == 1.0 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.connection.syn'] == 1.0 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.connection.synack'] == 1.0 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.connection.rst'] == 1.0 else "normal")
        predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.dstport'] in [80] else "normal")
        predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.flags'] in [18.0] else "normal")
        predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.options'] != '0' else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.seq'] > 1000000000.0 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.srcport'] == 80.0 else "normal")
        predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.len'] > 100 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.ack_raw'] < 1000000000.0 else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['tcp.payload'] != '0' else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['ip.src_host'] == "192.168.0.170" else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['ip.dst_host'] == "192.168.0.170" else "normal")
        # predicted_attack_types.append("normal" if dataset.iloc[i]['http.request.method'] in ["GET", "POST"] else "attack")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['http.request.uri.query'] == "0" else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['http.referer'] == "0" else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['http.request.full_uri'] == "0" else "normal")
        # predicted_attack_types.append("attack" if dataset.iloc[i]['udp.port'] == 80 else "normal")
        y_true.append(attack_type)
        y_pred.append(mode(predicted_attack_types))

c_report = classification_report(y_true, y_pred)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/result-llm-{sample_size}-2.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

Predicting attack entries...: 100%|███████████████████████████| 1000/1000 [00:00<00:00, 2604.96it/s]


              precision    recall  f1-score   support

      attack       0.85      0.25      0.39      1000
      normal       0.56      0.95      0.71      1000

    accuracy                           0.60      2000
   macro avg       0.70      0.60      0.55      2000
weighted avg       0.70      0.60      0.55      2000

[[252 748]
 [ 45 955]]


In [87]:
################################################################################
# Generate Rules with transposed data
################################################################################

import os
import dotenv
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze and identify patterns between normal and attack entries by comparing corresponding fields.
Generate 5 signature-based rules for top 5 important features to filter an entry as either normal or attack. 
Output only in the JSON format with the structure: 
{{'feature1': 'rule', 'feature2': 'rule', ..., 'feature5': 'rule'}}.

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```
"""
prompt = PromptTemplate(template=template, input_variables=["normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
model_name = "gpt-4o"
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)
# model_name = "gemini-1.5-pro"
# llm = ChatAnthropic(model='claude-3-opus-20240229')
# model_name = "claude-3-opus-20240229"
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")

normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=10)['documents'][0]

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=10)['documents'][0]

normal_entries = {}
for i, feature_name in enumerate(normal_df_train.columns.to_list()):
    normal_entries[feature_name] = [json.loads(doc.replace("'", "\""))[i] for doc in normal_documents]

attack_entries = {}
for i, feature_name in enumerate(attack_df_train.columns.to_list()):
    attack_entries[feature_name] = [json.loads(doc.replace("'", "\""))[i] for doc in attack_documents]

completion = chain.invoke({
    "feature_names": normal_df_train.columns.to_list(),
    "normal_entries": json.dumps(normal_entries),
    "attack_entries": json.dumps(attack_entries)
    })

print(completion.content)

with open(f"results/generated-rules-{sample_size}-llm-{model_name}.txt", "a") as f:
    f.write(completion.content)

c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


```json
{
  "ip.src_host": "if ip.src_host == '192.168.0.128' and ip.dst_host == '192.168.0.170' then 'attack' else 'normal'",
  "tcp.ack": "if tcp.ack in [1.0, 262.0, 122.0, 446.0, 320.0] then 'attack' else 'normal'",
  "tcp.dstport": "if tcp.dstport in [46616.0, 43030.0, 59964.0, 44672.0, 59716.0, 45208.0, 42468.0, 51916.0, 34892.0, 41524.0] then 'attack' else 'normal'",
  "tcp.flags": "if tcp.flags in [17.0, 18.0] then 'attack' else 'normal'",
  "tcp.srcport": "if tcp.srcport == '80.0' then 'attack' else 'normal'"
}
```


In [101]:
# Evaluate generated rule
################################################################################

from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

datasets = {"normal": normal_df_test, "attack": attack_df_test}
y_pred = []
y_true = []
for attack_type, dataset in datasets.items():
    test_set_size = dataset.shape[0]
    for i in tqdm(range(test_set_size), ncols=100, desc=f"Predicting {attack_type} entries..."):
        predicted_attack_types = []
        # predicted_attack_types.append("attack" if (dataset.iloc[i]['ip.src_host'] == "192.168.0.128") and (dataset.iloc[i]['ip.dst_host'] == "192.168.0.170") else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.srcport']) == "80.0" else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.flags']) == "18.0" else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.ack']) == "1.0" else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.dstport']) != "1883.0" else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['http.request.method']) not in ["6.0", "59.0"] else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.ack_raw']) not in ["1883.0", "60500.0", "58731.0"] else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.checksum']) == "18.0" else "normal")
        # predicted_attack_types.append("attack" if str(dataset.iloc[i]['tcp.options']) != "0" else "normal")
        predicted_attack_types.append("attack" if (dataset.iloc[i]['dns.qry.name.len']) == '0.0' else "normal")
        y_true.append(attack_type)
        y_pred.append(mode(predicted_attack_types))

c_report = classification_report(y_true, y_pred)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/result-llm-{sample_size}-2.txt", "a") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

Predicting attack entries...: 100%|██████████████████████████| 1000/1000 [00:00<00:00, 15588.56it/s]


              precision    recall  f1-score   support

      attack       1.00      1.00      1.00      1000
      normal       1.00      1.00      1.00      1000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

[[ 999    1]
 [   0 1000]]


In [90]:
normal_entries

{'frame.time': [' 2021 17:19:23.748023000 ',
  ' 2021 15:41:43.312269000 ',
  ' 2021 20:26:22.889420000 ',
  ' 2021 14:31:22.580423000 ',
  ' 2021 19:30:32.025270000 ',
  ' 2021 16:21:11.840867000 ',
  ' 2021 14:24:49.475107000 ',
  ' 2021 16:04:05.260009000 ',
  ' 2021 21:00:11.914284000 ',
  ' 2021 14:29:47.500017000 '],
 'ip.src_host': ['192.168.0.101',
  '192.168.0.128',
  '192.168.0.101',
  '192.168.0.101',
  '192.168.0.101',
  '192.168.0.101',
  '192.168.0.128',
  '192.168.0.101',
  '192.168.0.101',
  '192.168.0.101'],
 'ip.dst_host': ['192.168.0.128',
  '192.168.0.101',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.101',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128'],
 'arp.src.proto_ipv4': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 'icmp.checksum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'icmp.seq_le': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'http.file_data': ['0.0',
  '0.0',
  '0.0',
  '0.

In [91]:
attack_entries

{'frame.time': [' 2021 18:53:58.406720000 ',
  ' 2021 21:41:18.683777000 ',
  ' 2021 18:42:50.392712000 ',
  ' 2021 21:42:40.085690000 ',
  ' 2021 18:27:36.407423000 ',
  ' 2021 21:43:03.475400000 ',
  ' 2021 21:40:51.402154000 ',
  ' 2021 20:40:19.090082000 ',
  ' 2021 19:47:30.509174000 ',
  ' 2021 21:40:11.073360000 '],
 'ip.src_host': ['192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128',
  '192.168.0.128'],
 'ip.dst_host': ['192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170',
  '192.168.0.170'],
 'arp.src.proto_ipv4': ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 'icmp.checksum': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'icmp.seq_le': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 'http.file_data': ['0',
  '0.0',
  '0',
  '0.0',


In [2]:
################################################################################
# Generate Feature Importance
################################################################################

import dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze the differences between normal and attack entries by comparing corresponding fields.
Generate top 5 features that can be used to filter an entry as either normal or attack.
Output only in the Python list structure.

Feature Names:
```{feature_names}```

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```

Example output:
['feature', 'feature2', 'feature3', 'feature4', 'feature5']
"""
prompt = PromptTemplate(template=template, input_variables=["feature_names", "normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
# llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 10, "fetch_k": 10})
normal_documents = retriever.invoke(str(normal_df_test.iloc[0].to_list()), filter={"source": "edge-iiotset", "label": "normal"})
attack_documents = retriever.invoke(str(attack_df_test.iloc[0].to_list()), filter={"source": "edge-iiotset", "label": "attack"})
completion = chain.invoke({
    "feature_names": normal_df_train.columns.to_list(),
    "normal_entries": ",\n".join([f"{doc.page_content} --> {doc.metadata['label']}" for doc in normal_documents]),
    "attack_entries": ",\n".join([f"{doc.page_content} --> {doc.metadata['label']}" for doc in attack_documents])
    })
print(completion.content)

/opt/home/s4025371/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/home/s4025371/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


```python
['tcp.dstport', 'tcp.flags', 'tcp.seq', 'tcp.ack', 'tcp.len']
```


In [ ]:
################################################################################
# Get a Summary
################################################################################

import dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
import tiktoken     # https://github.com/openai/tiktoken

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
Given normal and attack network data entries, output human understandable small summary on 
how attack and normal entries can be simply separated.

Feature Names:
```{feature_names}```

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```
"""
prompt = PromptTemplate(template=template, input_variables=["feature_names", "normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
# llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5, "fetch_k": 5})
normal_documents = retriever.invoke(str(normal_df_test.iloc[0].to_list()), filter={"source": "edge-iiotset", "label": "normal"})
attack_documents = retriever.invoke(str(attack_df_test.iloc[0].to_list()), filter={"source": "edge-iiotset", "label": "attack"})
completion = chain.invoke({
    "feature_names": normal_df_train.columns.to_list(),
    "normal_entries": ",\n".join([f"{doc.page_content} --> {doc.metadata['label']}" for doc in normal_documents]),
    "attack_entries": ",\n".join([f"{doc.page_content} --> {doc.metadata['label']}" for doc in attack_documents])
    })
print(completion)

/opt/home/s4025371/anaconda3/envs/llm/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


content='Based on the provided data entries, we can see that normal entries have higher values for features such as flow duration, header length, rate, duration, and total size compared to attack entries. Additionally, normal entries have more occurrences of protocols like HTTP, HTTPS, DNS, SSH, TCP, UDP, DHCP, ARP, ICMP, and IPv.\n\nOn the other hand, attack entries have lower values for the mentioned features and do not have as many occurrences of the mentioned protocols. Attack entries also tend to have higher values for features like magnitude, radius, covariance, variance, and weight compared to normal entries.\n\nIn summary, normal entries exhibit higher network activity and a wider range of protocols, while attack entries show lower network activity and fewer protocol occurrences.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 149, 'prompt_tokens': 2787, 'total_tokens': 2936}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint':

In [ ]:
################################################################################
# 
################################################################################

import dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix

dotenv.load_dotenv()

template = """
Your task is to identify whether the query is attack or normal. Then 
generate a policy to filter the given query based on the values. 
You will be given feature names of the entries and similar entries along 
with the input query to make a decision.

Feature Names:
```{feature_names}```

Similar Entries:
```{similar_entries}```

Input Query: 
```{query}```

Policy:
"""
prompt = PromptTemplate(template=template, input_variables=["feature_names", "similar_entries", "query"])
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db-{train_set_size}-2")
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5, "fetch_k": 5})
query_document = str(normal_df_test.iloc[5].to_list())
similar_documents = retriever.invoke(query_document, filter={"source": "edge-iiotset"})
chain.invoke({
    "feature_names": normal_df_train.columns.to_list(),
    "similar_entries": ",\n".join([f"{doc.page_content} --> {doc.metadata['label']}" for doc in similar_documents]),
    "query": query_document
    })

In [ ]:
# print(completion.text)
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
num_tokens = len(encoding.encode(str(completion.text)))
print("Num tokens:", num_tokens)

Num tokens: 2780


In [77]:
################################################################################
# Generate Rules with transposed data
################################################################################

import os
import dotenv
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from statistics import mode
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

dotenv.load_dotenv(os.getcwd() + '/../.env')

template = """
You are provided with network data entries categorized as either normal or attack, along with their corresponding feature names.
Carefully analyze and identify patterns between normal and attack entries by comparing corresponding fields.
Generate 5 signature-based rules for top 5 important features to filter an entry as either normal or attack. 
Output only in the JSON format with the structure: 
{{'feature1': 'rule', 'feature2': 'rule', ..., 'feature5': 'rule'}}.

Normal Entries:
```{normal_entries}```

Attack Entries:
```{attack_entries}```
"""
prompt = PromptTemplate(template=template, input_variables=["normal_entries", "attack_entries"])
llm = ChatOpenAI(model="gpt-4o", temperature=0.0)
model_name = "gpt-4o"
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.0)
# model_name = "gemini-1.5-pro"
# llm = ChatAnthropic(model='claude-3-opus-20240229')
# model_name = "claude-3-opus-20240229"
chain = prompt | llm
train_set_size = sample_size
embeddings = HuggingFaceEmbeddings()
vector_store = Chroma(
    collection_name="edge-iiotset",
    embedding_function=embeddings, 
    persist_directory=f"./vector-stores/chroma-db3-{train_set_size}-2")
retriever = vector_store.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5, "fetch_k": 5})



# normal_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'normal'})['embeddings']
# normal_mean_vector = np.mean(normal_vectors, axis=0).tolist()
# normal_documents = vector_store._collection.query(query_embeddings=[normal_mean_vector], n_results=10)['metadatas'][0]

attack_vectors = vector_store._collection.get(include=['embeddings'], where={'label': 'attack'})['embeddings']
attack_mean_vector = np.mean(attack_vectors, axis=0).tolist()
attack_documents = vector_store._collection.query(query_embeddings=[attack_mean_vector], n_results=10)['metadatas'][0]

query_document = str(attack_mean_vector)
similar_documents = retriever.invoke(query_document, filter={'label': 'attack'})

# print([x['label'] for x in normal_documents])
print([x['label'] for x in attack_documents])
print([doc.metadata['label'] for doc in similar_documents])

c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


['attack', 'attack', 'attack', 'attack', 'attack', 'attack', 'attack', 'attack', 'attack', 'attack']
['normal', 'normal', 'normal', 'normal', 'normal']
